# AutoEncoder

We have taken a movie dataset and our task is to predict that the user has rated the movie or not using the AutoEncoder. The rating is from 1 to 5. AutoEncoder is Unsupervised Deep Learning algorithm, used for Recommended system.

In [1]:
#Importing Libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
#Importing the dataset
movies = pd.read_csv('ml-1m/movies.dat',sep = '::', header = None, engine = 'python', encoding = 'latin-1')
movies

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [3]:
users = pd.read_csv('ml-1m/users.dat',sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [4]:
ratings = pd.read_csv('ml-1m/ratings.dat',sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [5]:
#Preparing the trainig set
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561
...,...,...,...,...
79994,943,1067,2,875501756
79995,943,1074,4,888640250
79996,943,1188,3,888640250
79997,943,1228,3,888640275


In [6]:
training_set = np.array(training_set, dtype = 'int')
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [7]:
#Preparing the test
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set

,1,6,5,887431973
0,1,10,3,875693118
1,1,12,5,878542960
2,1,14,5,874965706
3,1,17,3,875073198
4,1,20,4,887431883
...,...,...,...,...
19994,458,648,4,886395899
19995,458,1101,4,886397931
19996,459,934,3,879563639
19997,460,10,3,882912371


In [8]:
#Converting the test set to numpy array
test_set = np.array(test_set, dtype = 'int')
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

In [9]:
#Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_users

943

In [10]:
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
nb_movies

1682

In [11]:
#Converting the data into array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0]==id_users]
        id_ratings = data[:, 2][data[:, 0]==id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [12]:
#Convert the data into torch tensor
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [15]:
# Creating the architecture of Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)
        

In [24]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('Epoch: '+str(epoch)+' Loss: '+ str(train_loss/s))

Epoch: 1 Loss: tensor(0.9003)
Epoch: 2 Loss: tensor(0.9014)
Epoch: 3 Loss: tensor(0.9010)
Epoch: 4 Loss: tensor(0.9006)
Epoch: 5 Loss: tensor(0.9001)
Epoch: 6 Loss: tensor(0.9000)
Epoch: 7 Loss: tensor(0.8994)
Epoch: 8 Loss: tensor(0.8996)
Epoch: 9 Loss: tensor(0.8983)
Epoch: 10 Loss: tensor(0.8978)
Epoch: 11 Loss: tensor(0.8974)
Epoch: 12 Loss: tensor(0.8969)
Epoch: 13 Loss: tensor(0.8968)
Epoch: 14 Loss: tensor(0.8960)
Epoch: 15 Loss: tensor(0.8953)
Epoch: 16 Loss: tensor(0.8955)
Epoch: 17 Loss: tensor(0.8949)
Epoch: 18 Loss: tensor(0.8943)
Epoch: 19 Loss: tensor(0.8939)
Epoch: 20 Loss: tensor(0.8934)
Epoch: 21 Loss: tensor(0.8933)
Epoch: 22 Loss: tensor(0.8925)
Epoch: 23 Loss: tensor(0.8929)
Epoch: 24 Loss: tensor(0.8915)
Epoch: 25 Loss: tensor(0.8918)
Epoch: 26 Loss: tensor(0.8908)
Epoch: 27 Loss: tensor(0.8910)
Epoch: 28 Loss: tensor(0.8898)
Epoch: 29 Loss: tensor(0.8897)
Epoch: 30 Loss: tensor(0.8889)
Epoch: 31 Loss: tensor(0.8887)
Epoch: 32 Loss: tensor(0.8888)
Epoch: 33 Loss: t

In [27]:
#Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('Test loss: '+str(test_loss/s))

Test loss: tensor(0.9541)


Our model has performed well on the test dataset. Got the test loss 0.95.